# Pandas dataframe to database 

In [1]:
import psycopg2 as ps
import pandas as pd

In [32]:
df = pd.read_csv('finalData_cleaned.csv')
df.shape

(2095, 14)

In [33]:
df.head()

,channel,video_id,title,upload_date,subs,views,likes,dislikes,like_perc,comments,tags,description,durations,player
0,Nate at StrataScratch,GGURenNfXI0,Multiple Approaches for Solving a Twitter Data...,2021-09-13,8900,1268,60,1,0.047319,23,"['Twitter Data Scientist', 'Twitter Data Scien...",We’ll closely examine one of the interesting T...,20.27,"{'embedHtml': '<iframe width=""480"" height=""270..."
1,Nate at StrataScratch,PlpUo6bHsBQ,Advanced Facebook Data Science SQL interview q...,2021-08-09,8900,3773,143,0,0.037901,30,"['facebook data science interview', 'facebook ...",This advanced SQL question is from the Faceboo...,24.27,"{'embedHtml': '<iframe width=""480"" height=""270..."
2,Nate at StrataScratch,uY2wfR8Dkqo,5 Tips for a Successful Data Science Interview,2021-07-26,8900,3738,139,1,0.037186,30,"['data science interview tips', 'data science ...",In this video of 5 tips for a successful data ...,20.20,"{'embedHtml': '<iframe width=""480"" height=""270..."
3,Nate at StrataScratch,yY7yau9j3xk,Data Science SQL Scenario Based Interview Ques...,2021-07-20,8900,2388,84,1,0.035176,16,"['sql interview question', 'facebook sql inter...",This Data Science SQL interview question is fr...,16.15,"{'embedHtml': '<iframe width=""480"" height=""270..."
4,Nate at StrataScratch,i-E4pdU2qXM,Data Science SQL Interview Question Walkthroug...,2021-07-13,8900,2162,107,0,0.049491,15,"['microsoft data science interview', 'microsof...",This Data Science SQL interview question is fr...,19.60,"{'embedHtml': '<iframe width=""480"" height=""270..."


In [34]:
df.rename(columns = {'id': 'video_id', 'title': 'video_title', 'subscriber_count': 'subs', 'view_count': 'views',
                    'like_count': 'likes', 'dislike_count': 'dislikes', "comment_count": "comments", "durations": "duration"}, inplace = True)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2095 entries, 0 to 2094
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   channel      2095 non-null   object 
 1   video_id     2095 non-null   object 
 2   video_title  2095 non-null   object 
 3   upload_date  2095 non-null   object 
 4   subs         2095 non-null   int64  
 5   views        2095 non-null   int64  
 6   likes        2095 non-null   int64  
 7   dislikes     2095 non-null   int64  
 8   like_perc    2095 non-null   float64
 9   comments     2095 non-null   int64  
 10  tags         2095 non-null   object 
 11  description  2095 non-null   object 
 12  duration     2095 non-null   float64
 13  player       2095 non-null   object 
dtypes: float64(2), int64(5), object(7)
memory usage: 229.3+ KB


## Necessary actions for dataframe upload to database
- Creating table in database
- **Checking** if the video exists in the database
    - If yes, **update** the corresponding video statistics 
    - If no, **insert** newly available video details as rows 

In [18]:
# create table 

def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS video_data(
                    channel VARCHAR(255),
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT,
                    upload_date DATE,
                    subs INTEGER,
                    views INTEGER,
                    likes INTEGER, 
                    dislikes INTEGER,
                    like_perc FLOAT,
                    comments INTEGER,
                    tags TEXT, 
                    description TEXT, 
                    duration FLOAT, 
                    player TEXT)""")
    curr.execute(create_table_command)

In [8]:
# checking if video already exists in table
def check_if_vid_exists(curr, video_id):
    query = ("""SELECT video_id FROM video_data WHERE video_id = %s""")
    curr.execute(query, [video_id])
    
    return curr.fetchone() is not None # if a row exists with the required video_id it returns the row, else returns None => video doesn't exist

In [9]:
# update row if video exists
def update_row(curr, video_id, video_title, subs, views, likes, dislikes, like_perc, comments):
    query = ("""UPDATE video_data SET video_title = %s,
                subs = %s,
                views = %s,
                likes = %s,
                dislikes = %s,
                like_perc = %s,
                comments = %s
                WHERE video_id = %s""")

    vars_to_update = (video_title, [subs, views, likes, dislikes, like_perc, comments, video_id])
    curr.execute(query, vars_to_update)

In [10]:
def update_db(curr, df):
    temp = pd.DataFrame(columns = df.columns)

    for i, data in df.iterrows():
        if check_if_vid_exists(curr, data['video_id']): # if video exists in table, we update the table with latest available statistics
            update_row(curr, data['video_id'], data['video_title'], data['subs'], data['views'], data['likes'],
                      data['dislikes'], data['like_perc'], data['comments'])

        else: # when new video is published, we insert it into table
            temp = temp.append(data)
    return temp

In [11]:
# insert command
def insert_row(curr, channel, video_id, video_title, upload_date, subs, views, likes, dislikes, like_perc, comments, tags, description, duration, player):
    insert_into_videos = ("""INSERT INTO video_data (channel, video_id, video_title, upload_date, subs, views, likes, dislikes, like_perc, comments, tags, description, duration, player)
                            VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""")
    rows_to_insert = (channel, video_id, video_title, upload_date, subs, views, likes, dislikes, like_perc, comments, tags, description, duration, player)
    curr.execute(insert_into_videos, rows_to_insert)

In [12]:
def df_to_db(curr, df):
    for i, data in df.iterrows():
        insert_row(curr, data['channel'], data['video_id'], data['video_title'], data['upload_date'], data['subs'], data['views'],
           data['likes'], data['dislikes'], data['like_perc'], data['comments'], data['tags'], data['description'],
           data['duration'], data['player'])

## Connecting to database and creating table

In [7]:
host_name = 
dbname = 
port = '5432'
username = 
password = 

In [36]:
# connecting to PostGres database locally

def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host = host_name, database = dbname, user = username, password = password, port = port)
    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn

### Creating table

In [20]:
conn = connect_to_db(host_name, dbname, port, username, password)

Connected!


In [21]:
curr = conn.cursor() # allows python code to execute sql commands in the database session

In [22]:
create_table(curr)

In [23]:
conn.commit()

### Update

In [40]:
conn = connect_to_db(host_name, dbname, port, username, password)

Connected!


In [41]:
curr = conn.cursor()

In [42]:
# currently database is empty 
new_vid_df = update_db(curr, df)

In [43]:
print(len(new_vid_df))

2095


In [44]:
conn.commit()

In [45]:
### 

In [46]:
conn = connect_to_db(host_name, dbname, port, username, password)

Connected!


In [47]:
curr = conn.cursor()

In [48]:
df_to_db(curr, new_vid_df)

In [49]:
conn.commit()